In [226]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC

from nltk import word_tokenize
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix,hstack

PATH_TO_DATA="/notebooks/roman/task1/data/"

import warnings
warnings.filterwarnings('ignore')

import re
from textblob import TextBlob

In [474]:
def submit(pred,name="ans"):
    ans=pd.read_csv(PATH_TO_DATA+"sample_submission.csv")
    ans["label"]=pred
    ans.to_csv("answers/"+name+".csv",index=False)

_____

### План
* tfidf
* tfidf c ngram stop words etc
* Count wectorizer
* линейные модели
* knn

### Добавить фичи
* Добавить теги
* Добавить дину подписи
* Добавить длину подписи деленную на n (n подобрать)
* Number of words
* Number of characters
* Average word length
* Number of stopwords
* Number of special characters
* Number of numerics
* Number of uppercase words

## tfidf
baseline

In [12]:
data_train=pd.read_csv(PATH_TO_DATA+"train_data.csv")
data_test=pd.read_csv(PATH_TO_DATA+"test_data.csv")

In [13]:
data_train_NoImg=data_train.drop("image_name",axis=1)
data_test_NoImg=data_test.drop("image_name",axis=1)

In [14]:
data_train_NoImg['label'].value_counts()

0    128292
1      8265
Name: label, dtype: int64

In [15]:
train,test=train_test_split(data_train_NoImg,random_state=42,test_size=0.3)

In [16]:
#tfidf
tfidf=TfidfVectorizer()
train_tfidf=tfidf.fit_transform(train["text"])
test_tfidf=tfidf.transform(test["text"])

In [17]:
#lr
lr=LogisticRegression(random_state=42)
lr.fit(train_tfidf,train["label"])

predict=lr.predict_proba(test_tfidf)[:,1]

In [18]:
roc_auc_score(test["label"],predict)

0.6019956360477763

### tfidf  stop words

In [22]:
stop_words = set(stopwords.words('english'))

In [24]:
#tfidf
tfidf=TfidfVectorizer(stop_words=stop_words)
train_tfidf=tfidf.fit_transform(train["text"])
test_tfidf=tfidf.transform(test["text"])

In [25]:
#lr
lr=LogisticRegression(random_state=42)
lr.fit(train_tfidf,train["label"])

predict=lr.predict_proba(test_tfidf)[:,1]

In [26]:
roc_auc_score(test["label"],predict)

0.5936288302634535

ухудшило

### tfidf c ngram

In [63]:
#tfidf
tfidf=TfidfVectorizer(ngram_range=(1,2))
train_tfidf=tfidf.fit_transform(train["text"])
test_tfidf=tfidf.transform(test["text"])

In [64]:
#lr
lr=LogisticRegression(random_state=42)
lr.fit(train_tfidf,train["label"])

predict=lr.predict_proba(test_tfidf)[:,1]

In [65]:
roc_auc_score(test["label"],predict)

0.6088818325159076

чутка улучшило

### lenth comment

In [67]:
#lenth feature
train["lenth"]=train["text"].apply(lambda x: len(x))
test["lenth"]=test["text"].apply(lambda x: len(x))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [69]:
tfidf=TfidfVectorizer(ngram_range=(1,2))
train_tfidf=tfidf.fit_transform(train["text"])
test_tfidf=tfidf.transform(test["text"])

In [71]:
a=csr_matrix(train["lenth"].values.reshape(len(train),-1))
b=csr_matrix(test["lenth"].values.reshape(len(test),-1))

In [72]:
train_tfidf_leth=hstack((train_tfidf,a))
test_tfidf_leth=hstack((test_tfidf,b))

In [77]:
lr=LogisticRegression(random_state=42)
lr.fit(train_tfidf_leth,train["label"])

predict=lr.predict_proba(test_tfidf_leth)[:,1]

In [78]:
roc_auc_score(test["label"],predict)

0.6088777231969127

чутка улучшило

### Count vectorizer

In [83]:
#lenth feature
train["lenth"]=train["text"].apply(lambda x: len(x))
test["lenth"]=test["text"].apply(lambda x: len(x))

a=csr_matrix(train["lenth"].values.reshape(len(train),-1))
b=csr_matrix(test["lenth"].values.reshape(len(test),-1))

In [84]:
tfidf=TfidfVectorizer(ngram_range=(1,2))
train_tfidf=tfidf.fit_transform(train["text"])
test_tfidf=tfidf.transform(test["text"])

In [85]:
counVec=CountVectorizer(ngram_range=(1,2))
train_counVec=counVec.fit_transform(train["text"])
test_counVec=counVec.transform(test["text"])

In [86]:
train_tfidf_leth_CV=hstack((train_tfidf,train_counVec,a))
test_tfidf_leth_CV=hstack((test_tfidf,test_counVec,b))

In [87]:
lr=LogisticRegression(random_state=42)
lr.fit(train_tfidf_leth_CV,train["label"])

predict=lr.predict_proba(test_tfidf_leth_CV)[:,1]

In [88]:
roc_auc_score(test["label"],predict)

0.6005282787690971

Count_vect ухучшил модель

_________-

# Try 2

### feature engenering
* Добавить теги done
* Добавить дину подписи done
* Добавить длину тега done
* Number of words done
* Average word length done
* Number of stopwords done

In [151]:
#Загрузка данных
data_train=pd.read_csv(PATH_TO_DATA+"train_data.csv")
data_test=pd.read_csv(PATH_TO_DATA+"test_data.csv")
casts=pd.read_csv(PATH_TO_DATA+"casts.csv",header=None)

In [152]:
# Теги картинок
imageTags={}
for i in casts.values:
    imageTags[i[0]]=i[2]

In [154]:
# Добавление столбца тегов картинки
data_train["tags"]=data_train["image_name"].replace(imageTags)

# Удаление номера картинки
data_train_NoImg=data_train.drop("image_name",axis=1)

In [157]:
#Приведение к нижнему регистру
data_train_NoImg["text"]=data_train_NoImg["text"].apply(lambda x:x.lower())
data_train_NoImg["tags"]=data_train_NoImg["tags"].apply(lambda x:x.lower())

In [186]:
# Flag  вопросительный и восклицательный знак
data_train_NoImg["special character"]=data_train_NoImg["text"].apply(lambda x:int(any([i in ["?","!"]for i in x])))

In [206]:
#Удаление згаков преинания
data_train_NoImg["text"]=data_train_NoImg["text"].apply(lambda x:re.sub(r'[^\w\s]','',x))

In [208]:
# Добавить дину подписи
# Добавить длину тега
data_train_NoImg["lenth_text"]=data_train_NoImg["text"].apply(lambda x: len(x))
data_train_NoImg["lenth_tags"]=data_train_NoImg["tags"].apply(lambda x: len(x))

In [212]:
# Number of words
data_train_NoImg["num_w_text"]=data_train_NoImg["text"].apply(lambda x: len(x.split(" ")))
data_train_NoImg["num_w_tags"]=data_train_NoImg["tags"].apply(lambda x: len(x.split(":")))

In [210]:
# Average word length
data_train_NoImg["average_w_lenth"]=data_train_NoImg["text"].apply(lambda x: np.mean([len(i) for i in x.split(" ")]))
data_train_NoImg["average_tag_lenth"]=data_train_NoImg["tags"].apply(lambda x: np.mean([len(i) for i in x.split(" ")]))

In [221]:
# Number of stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')


data_train_NoImg["num_stop_w"]=data_train_NoImg["text"].apply(lambda x: sum([int(w in stop) for w in x.split(" ")]))

In [238]:
# Spelling correction (минут 40 считалось)
data_train_NoImg["corrected_text"]=data_train_NoImg["text"].apply(lambda x: str(TextBlob(x).correct()))

In [246]:
#Flag была ли ошибка (минут 15)
data_train_NoImg["error_in_text"]=data_train_NoImg["corrected_text"].apply(lambda x: int(x in data_train_NoImg["text"].values))

In [262]:
#Обработка тегов
data_train_NoImg["tags_new"]=data_train_NoImg["tags"].apply(lambda x:" ".join(x.replace(" ","_").split(":,")))

In [265]:
# Stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()

data_train_NoImg["stemm_text"]=data_train_NoImg["text"].apply(lambda x: " ".join([st.stem(i) for i in x.split(" ")]))

In [266]:
# Lemmatization
from textblob import Word
data_train_NoImg["lemm_text"]=data_train_NoImg["text"].apply(lambda x: " ".join([Word(i).lemmatize() for i in x.split(" ")]))

### Create model

In [354]:
train, test, y_train, y_test=train_test_split(data_train_NoImg.drop("label",axis=1),data_train_NoImg["label"],random_state=42,test_size=0.3)
train.head(1)

,text,tags,special character,lenth_text,lenth_tags,num_w_text,num_w_tags,average_w_lenth,average_tag_lenth,num_stop_w,corrected_text,error_in_text,tags_new,stemm_text,lemm_text
9091,mom but thats a letter,"facial expression:,emotion:,smile:,event:,chri...",0,22,52,5,6,3.6,25.5,2,mon but that a letter,0,facial_expression emotion smile event christmas:,mom but that a letter,mom but thats a letter


In [355]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics import roc_auc_score
from scipy.sparse import csr_matrix,hstack

### default    text and tags

In [356]:
#tfidf
tfidf=TfidfVectorizer(ngram_range=(1,2))

text_tfidf_train=tfidf.fit_transform(train["text"])
text_tfidf_test=tfidf.transform(test["text"])

tags_tfidf_train=tfidf.fit_transform(train["tags"])
tags_tfidf_test=tfidf.transform(test["tags"])

In [357]:
not_str=["special character","lenth_text","lenth_tags",
          "num_w_text","num_w_tags","average_w_lenth",
          "average_tag_lenth","num_stop_w","error_in_text"]

In [358]:
#Численные признаки в разряженный вид
train_num_csr=csr_matrix(train[not_str].values.reshape(len(train),-1))
test_num_csr=csr_matrix(test[not_str].values.reshape(len(test),-1))

In [359]:
# Обьеденение
train_csr=hstack((text_tfidf_train,tags_tfidf_train,train_num_csr))
test_csr=hstack((text_tfidf_test,tags_tfidf_test,test_num_csr))

In [360]:
lr=LogisticRegression(random_state=42)
lr.fit(train_csr,y_train)
predict=lr.predict_proba(test_csr)[:,1]
roc_auc_score(y_test,predict)

0.7286983563787488

### correct_text and tags_new

In [314]:
#tfidf
tfidf=TfidfVectorizer(ngram_range=(1,2))

text_tfidf_train=tfidf.fit_transform(train["corrected_text"])
text_tfidf_test=tfidf.transform(test["corrected_text"])

tags_tfidf_train=tfidf.fit_transform(train["tags_new"])
tags_tfidf_test=tfidf.transform(test["tags_new"])

In [315]:
#Численные признаки в разряженный вид
train_num_csr=csr_matrix(train[not_str].values.reshape(len(train),-1))
test_num_csr=csr_matrix(test[not_str].values.reshape(len(test),-1))

In [316]:
# Обьеденение
train_csr=hstack((text_tfidf_train,tags_tfidf_train,train_num_csr))
test_csr=hstack((text_tfidf_test,tags_tfidf_test,test_num_csr))

In [317]:
lr=LogisticRegression(random_state=42)
lr.fit(train_csr,y_train)
predict=lr.predict_proba(test_csr)[:,1]
roc_auc_score(y_test,predict)

0.7309938756588836

### correct_text and tags_new stemm lemm

In [489]:
#tfidf
tfidf=TfidfVectorizer(ngram_range=(1,2))

# text_tfidf_train=tfidf.fit_transform(train["corrected_text"])
# text_tfidf_test=tfidf.transform(test["corrected_text"])

tags_tfidf_train=tfidf.fit_transform(train["tags_new"])
tags_tfidf_test=tfidf.transform(test["tags_new"])

# stem_tfidf_train=tfidf.fit_transform(train["stemm_text"])
# stem_tfidf_test=tfidf.transform(test["stemm_text"])

lem_tfidf_train=tfidf.fit_transform(train["lemm_text"])
lem_tfidf_test=tfidf.transform(test["lemm_text"])

In [339]:
#Численные признаки в разряженный вид
train_num_csr=csr_matrix(train[not_str].values.reshape(len(train),-1))
test_num_csr=csr_matrix(test[not_str].values.reshape(len(test),-1))

In [340]:
# Обьеденение
# train_csr=hstack((text_tfidf_train,tags_tfidf_train,train_num_csr,stem_tfidf_train,lem_tfidf_train))
# test_csr=hstack((text_tfidf_test,tags_tfidf_test,test_num_csr,stem_tfidf_test,lem_tfidf_test))

train_csr=hstack((tags_tfidf_train,train_num_csr,lem_tfidf_train))
test_csr=hstack((tags_tfidf_test,test_num_csr,lem_tfidf_test))

In [341]:
lr=LogisticRegression(random_state=42)
lr.fit(train_csr,y_train)
predict=lr.predict_proba(test_csr)[:,1]
roc_auc_score(y_test,predict)

0.7312916360872588

_____

## Predict for submit

In [366]:
# Добавление столбца тегов картинки
data_test["tags"]=data_test["image_name"].replace(imageTags)

# Удаление номера картинки
data_test_NoImg=data_test.drop("image_name",axis=1)

In [367]:
#Приведение к нижнему регистру
data_test_NoImg["text"]=data_test_NoImg["text"].apply(lambda x:x.lower())
data_test_NoImg["tags"]=data_test_NoImg["tags"].apply(lambda x:x.lower())

In [368]:
# Flag  вопросительный и восклицательный знак
data_test_NoImg["special character"]=data_test_NoImg["text"].apply(lambda x:int(any([i in ["?","!"]for i in x])))

In [369]:
#Удаление згаков преинания
data_test_NoImg["text"]=data_test_NoImg["text"].apply(lambda x:re.sub(r'[^\w\s]','',x))

In [370]:
# Добавить дину подписи
# Добавить длину тега
data_test_NoImg["lenth_text"]=data_test_NoImg["text"].apply(lambda x: len(x))
data_test_NoImg["lenth_tags"]=data_test_NoImg["tags"].apply(lambda x: len(x))

In [371]:
# Number of words
data_test_NoImg["num_w_text"]=data_test_NoImg["text"].apply(lambda x: len(x.split(" ")))
data_test_NoImg["num_w_tags"]=data_test_NoImg["tags"].apply(lambda x: len(x.split(":")))

In [372]:
# Average word length
data_test_NoImg["average_w_lenth"]=data_test_NoImg["text"].apply(lambda x: np.mean([len(i) for i in x.split(" ")]))
data_test_NoImg["average_tag_lenth"]=data_test_NoImg["tags"].apply(lambda x: np.mean([len(i) for i in x.split(" ")]))

In [374]:
# Number of stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')

data_test_NoImg["num_stop_w"]=data_test_NoImg["text"].apply(lambda x: sum([int(w in stop) for w in x.split(" ")]))

In [375]:
# Spelling correction
data_test_NoImg["corrected_text"]=data_test_NoImg["text"].apply(lambda x: str(TextBlob(x).correct()))

In [378]:
#Flag была ли ошибка
data_test_NoImg["error_in_text"]=data_test_NoImg["corrected_text"].apply(lambda x: int(x in data_test_NoImg["text"].values))

In [379]:
#Обработка тегов
data_test_NoImg["tags_new"]=data_test_NoImg["tags"].apply(lambda x:" ".join(x.replace(" ","_").split(":,")))

In [380]:
# Stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()

data_test_NoImg["stemm_text"]=data_test_NoImg["text"].apply(lambda x: " ".join([st.stem(i) for i in x.split(" ")]))

In [381]:
# Lemmatization
from textblob import Word
data_test_NoImg["lemm_text"]=data_test_NoImg["text"].apply(lambda x: " ".join([Word(i).lemmatize() for i in x.split(" ")]))

### creat final model and predict

In [482]:
X_train=data_train_NoImg.drop("label",axis=1)
y=data_train_NoImg["label"]

In [483]:
X_test=data_test_NoImg.drop("id",axis=1)

In [484]:
#tfidf
tfidf=TfidfVectorizer(ngram_range=(1,2))

tags_tfidf_train=tfidf.fit_transform(X_train["tags_new"])
tags_tfidf_test=tfidf.transform(X_test["tags_new"])


lem_tfidf_train=tfidf.fit_transform(X_train["lemm_text"])
lem_tfidf_test=tfidf.transform(X_test["lemm_text"])

In [485]:
#Численные признаки в разряженный вид
train_num_csr=csr_matrix(X_train[not_str])
test_num_csr=csr_matrix(X_test[not_str])

In [486]:
# Обьеденение
train_csr=hstack((tags_tfidf_train,train_num_csr,lem_tfidf_train))
test_csr=hstack((tags_tfidf_test,test_num_csr,lem_tfidf_test))

In [487]:
lr=LogisticRegression(random_state=42)
lr.fit(train_csr,y)
predict=lr.predict_proba(test_csr)[:,1]

In [488]:
submit(predict,"1")